In [ ]:
import numpy as np

def phi(x):
    return 1 / (1+np.exp(-x))

def phi1(phi): # returns dphi/dx given phi(x)
    return phi*(1-phi)

class Node:
    def __init__(self,dimensions):
        self.w = np.zeros((1,dimensions))
    def evaluate(self, x):
        return phi( x @ self.w.T)
    def refine(self,x,yhat):
        y = self.evaluate(x)
        e = y - yhat
        correction = (x.T @ (phi1(y)*e)).T
        self.w = self.w - correction
    def calibrate(self, x, yhat, iterations):
        for counter in range(0,iterations):
            self.refine(x,yhat)
        
n = Node(3)
x = np.array([[1,0,0],[1,0,1],[1,1,0],[1,1,1]])
yhat = np.array([[0,0,0,1]]).T
n.calibrate(x,yhat,1000)
n.evaluate(x) - yhat

In [ ]:
class Layer:
    def __init__(self,dimensions,classes):
        self.nodes = []
        for i in range(0,classes):
            self.nodes.append(Node(dimensions))
    
    def evaluate(self,x):
        y = np.empty((x.shape[0],0))
        for node in self.nodes:
            y = np.concatenate((y,node.evaluate(x)),axis=1)
        return y
    
    def calibrate(self,x,yhat,iterations):
        j=0
        for node in self.nodes:
            node.calibrate(x,yhat[0:,j:j+1],iterations)
            j=j+1
            
nn = Layer(3,3)
x = np.array([[1,0,0],[1,0,1],[1,1,0],[1,1,1]])
yhat = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,1]]).T
nn.calibrate(x,yhat,100)
np.round(nn.evaluate(x)).astype(int) - yhat